In [1]:
import tensorflow as tf
print(tf.__version__)
import itertools

2024-12-18 16:29:50.676375: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-18 16:29:50.717925: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-18 16:29:51.362708: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.13.1


In [2]:
import tensorflow as tf

# Create a constant tensor
hello = tf.constant('Hello, TensorFlow!')

# Run the TensorFlow session
tf.print(hello)


Hello, TensorFlow!


2024-12-18 16:29:52.123146: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-18 16:29:52.172308: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv('AI_Human.csv')

# Display the first few rows of the DataFrame
df.head()


,text,generated
0,Cars. Cars have been around since they became ...,0.0
1,Transportation is a large necessity in most co...,0.0
2,"""America's love affair with it's vehicles seem...",0.0
3,How often do you ride in a car? Do you drive a...,0.0
4,Cars are a wonderful thing. They are perhaps o...,0.0


In [4]:
# Importing necessary libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical operations
import matplotlib.pyplot as plt  # For plotting graphs
from sklearn.model_selection import train_test_split  # For splitting the dataset
from keras.preprocessing.sequence import pad_sequences  # For padding sequences
from keras.models import Sequential  # For creating a sequential model
from keras.layers import Dense, LSTM, Embedding, SpatialDropout1D  # For building LSTM model
from tensorflow.keras.preprocessing.text import Tokenizer
import openpyxl
print("openpyxl installed and working correctly!")


openpyxl installed and working correctly!


In [7]:
# Separate features and labels
texts = df.iloc[:, 0].values  # The text column
labels = df.iloc[:, 1].values  # The labels column (0 for human, 1 for AI)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)


# Separate features and labels
texts = df.iloc[:, 0].values  # The text column
labels = df.iloc[:, 1].values  # The labels column (0 for human, 1 for AI)

# Split the data into training and temp sets (80% train, 20% temp)
X_train, X_temp, y_train, y_temp = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Split the temp set into validation and test sets (50% of temp for each, so 10% of original)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:
from tqdm import tqdm
import os
import pandas as pd
import itertools
from tqdm import tqdm
import numpy as np
import pickle

model_path = "monModel.h5"

def train_model(dropout, learning_rate, batch_size, vocab_size, X_train_pad, X_val_pad, X_test_pad, y_train, y_val, y_test):
    # Building the LSTM model
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=128))
    model.add(SpatialDropout1D(0.2))
    model.add(LSTM(100, dropout=dropout, recurrent_dropout=0.2))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model with specified learning rate
    model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_pad, y_train, 
                        validation_data=(X_val_pad, y_val), 
                        epochs=10,  # Adjust the number of epochs as needed
                        batch_size=batch_size, 
                        verbose=1)  # Display training progress
    
     # Save the model
    model.save(model_path)
    print(f"Model saved to {model_path}")
    

    # Get validation accuracy
    val_accuracy = history.history['val_accuracy'][-1]

    # Evaluate the model on test data
    test_loss, test_accuracy = model.evaluate(X_test_pad, y_test, verbose=0)

    return val_accuracy, test_accuracy


# Paramètres
dropout_list = [0,0.1]
learning_rate_list = [0.3]
batch_size_list = [64,128]
number_of_token_in_vocab = [500,1000,2000]
excel_file = "model_accuracy_results.xlsx"

# Déterminer toutes les combinaisons possibles
all_combinations = list(itertools.product(dropout_list, learning_rate_list, batch_size_list, number_of_token_in_vocab))

# Lire le fichier Excel s'il existe
if os.path.exists(excel_file):
    existing_results = pd.read_excel(excel_file)
    completed_combinations = set(zip(
        existing_results['Dropout'],
        existing_results['Learning Rate'],
        existing_results['Batch Size'],
        existing_results['Vocab Size']
    ))
else:
    existing_results = pd.DataFrame()
    completed_combinations = set()

# Filtrer les combinaisons restantes
remaining_combinations = [comb for comb in all_combinations if comb not in completed_combinations]
print(remaining_combinations)

# Initialiser la barre de progression
with tqdm(total=len(remaining_combinations) * 5, desc="Training Models", unit="run") as pbar:
    for (dropout, learning_rate, batch_size, vocab_size) in remaining_combinations:
        print(f"Running combination: Dropout={dropout}, Learning Rate={learning_rate}, Batch Size={batch_size}, Vocab Size={vocab_size}")
        val_accuracies = []
        test_accuracies = []

        # Tokenizing the text data
        tokenizer = Tokenizer(num_words=vocab_size)
        tokenizer.fit_on_texts(X_train)
        # Save the tokenizer for later use
        with open("tokenizer.pkl", "wb") as f:
            pickle.dump(tokenizer, f)
        print("Tokenizer saved to tokenizer.pkl")
        X_train_seq = tokenizer.texts_to_sequences(X_train)
        X_val_seq = tokenizer.texts_to_sequences(X_val)
        X_test_seq = tokenizer.texts_to_sequences(X_test)
        print("Fin de la tokenisation")
        max_sequence_length = 1000
        X_train_pad = pad_sequences(X_train_seq, maxlen=max_sequence_length)
        X_val_pad = pad_sequences(X_val_seq, maxlen=max_sequence_length)
        X_test_pad = pad_sequences(X_test_seq, maxlen=max_sequence_length)

        # Exécuter chaque combinaison 5 fois
        for i in range(5):
            print("entrainement du model : ", dropout, learning_rate, batch_size, vocab_size)
            val_accuracy, test_accuracy = train_model(
                dropout, learning_rate, batch_size, vocab_size,
                X_train_pad, X_val_pad, X_test_pad,
                y_train, y_val, y_test
            )
            val_accuracies.append(val_accuracy)
            test_accuracies.append(test_accuracy)
            pbar.update(1)

        avg_val_accuracy = np.mean(val_accuracies)
        avg_test_accuracy = np.mean(test_accuracies)

        # Ajouter les résultats pour cette combinaison
        new_result = {
            'Dropout': dropout,
            'Learning Rate': learning_rate,
            'Batch Size': batch_size,
            'Vocab Size': vocab_size,
            'Average Validation Accuracy': avg_val_accuracy,
            'Average Test Accuracy': avg_test_accuracy
        }

        # Ajouter au DataFrame existant
        existing_results = pd.concat([existing_results, pd.DataFrame([new_result])], ignore_index=True)

        # Sauvegarder immédiatement dans le fichier Excel
        existing_results.to_excel(excel_file, index=False)

print("All results saved to 'model_accuracy_results.xlsx'")

[(0.1, 0.3, 64, 2000)]


Training Models:   0%|                                   | 0/5 [00:00<?, ?run/s]

Running combination: Dropout=0.1, Learning Rate=0.3, Batch Size=64, Vocab Size=2000
Tokenizer saved to tokenizer.pkl
Fin de la tokenisation
entrainement du model :  0.1 0.3 64 2000
Epoch 1/5
6091/6091 [==============================] - 2855s 469ms/step - loss: 0.0840 - accuracy: 0.9723 - val_loss: 0.0161 - val_accuracy: 0.9954
Epoch 2/5
6091/6091 [==============================] - 2720s 447ms/step - loss: 0.0144 - accuracy: 0.9957 - val_loss: 0.0081 - val_accuracy: 0.9974
Epoch 3/5
6091/6091 [==============================] - 2731s 448ms/step - loss: 0.0072 - accuracy: 0.9978 - val_loss: 0.0050 - val_accuracy: 0.9986
Epoch 4/5
6091/6091 [==============================] - 2738s 449ms/step - loss: 0.0046 - accuracy: 0.9986 - val_loss: 0.0040 - val_accuracy: 0.9990
Epoch 5/5
6091/6091 [==============================] - ETA: 0s - loss: 0.0033 - accuracy: 0.9989